# 2025 March Madness Kaggle Competition

- Goal of the Competition

이 컴피티션의 목표는 과거 데이터를 바탕으로 NCAA 디비전 1 대학농구 토너먼트, 혹은 "3월의 광란"의 결과를 예측하는 것이다.

- Submission Strategy

정규 시즌 결과를 바탕으로 팀별 승률, 평균 점수차(마진), 평균 득점 등을 구한 뒤 이를 바틍으로 토너먼트 결과를 예측한다. 학습 데이터는 2024년까지의 토너먼트 결과를 사용하기로 한다.

예측 방법으로는 XGBoost 모형을 사용하여 각 특성을 바탕으로 마진을 구하는 모형을 생성한 뒤, 예측값(마진)을 승률로 변환하는 방식을 사용할 것이다.

## 1. Import Python packages

In [1]:
# import numpy as np
# import pandas as pd

## 2. Explore the Data

In [2]:
# # Regular Season Results
# regular_season_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
# regular_season_results['margin'] = regular_season_results['WScore'] - regular_season_results['LScore']

# rs_detailed = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv')
# df_rs_games = pd.merge(left=regular_season_results, right=rs_detailed)

# df_rs_games

## 3. Data Preprocessing

### 3-1. Margin of Each Game by Team

In [3]:
# W_columns = {'WTeamID': 'TeamID', 'WScore': 'Score', 'WLoc':'Loc', 
#              'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA',
#              'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast', 'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF'}

# L_columns = {'LTeamID': 'TeamID', 'LScore': 'Score', 'WLoc':'Loc',
#              'LFGM': 'FGM', 'LFGA': 'FGA', 'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA',
#              'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast', 'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF'}

# W_results = df_rs_games[['Season', 'DayNum', 'WTeamID', 'WScore', 'WLoc', 'margin', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
#      'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']] \
#     .rename(columns=W_columns, inplace=False)
# W_results['Result'] = 1

# L_results = df_rs_games[['Season', 'DayNum', 'LTeamID', 'LScore', 'WLoc', 'margin', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
#      'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']] \
#     .rename(columns=L_columns, inplace=False)

# # change the sign of the margin
# L_results['margin'] = - L_results['margin']
# L_results['Result'] = 0

# regular_season_results_flatten = pd.concat([W_results, L_results], axis=0)
# regular_season_results_flatten.drop('Loc', axis=1, inplace=True)

# regular_season_results_flatten

### 3-2. Margin of Each Team by Season

In [4]:
# # Total
# summary_sum = regular_season_results_flatten.groupby(by=['Season', 'TeamID'], as_index=False) \
#                                             .sum()
# # Mean
# summary_mean = regular_season_results_flatten.groupby(by=['Season', 'TeamID'], as_index=False) \
#                                             .mean()


# # FG%
# summary_mean.columns

# summary_mean['FG%'] = summary_mean.FGM / summary_mean.FGA
# summary_mean['3P%'] = summary_mean.FGM3 / summary_mean.FGA3
# summary_mean['FT%'] = summary_mean.FTM / summary_mean.FTA

# summary_mean.rename(columns={'Result':'Win%'}, inplace=True)

# summary_mean

### 3-3. Scaling

In [5]:
# from sklearn.preprocessing import MinMaxScaler

# regular_season_results_by_team = regular_season_results_flatten.groupby(by=['Season', 'TeamID'],
#                                                                         as_index=False) \
#                                                                .margin \
#                                                                .mean() \
#                                                                .sort_values(by='Season',
#                                                                             ascending=False)
# regular_season_results_by_team
# 
# # Min-Max scaling
# margin_min = regular_season_results_by_team.groupby('Season').margin.transform('min')
# margin_max = regular_season_results_by_team.groupby('Season').margin.transform('max')

# regular_season_results_by_team['margin_scale'] = (regular_season_results_by_team['margin'] - margin_min) / (margin_max - margin_min) 

# regular_season_results_by_team

## 4. Prediction

In [6]:
# # Create train data X
# df_X = pd.merge(left=summary_mean,
#                          right=summary_mean, 
#                          how='left', on=['Season'])
# df_X = df_X[df_X['TeamID_x'] < df_X['TeamID_y']]

# # Labeling
# # df_X['ID'] = df_X['Season'].astype(str) \
#            + '_' + df_X['TeamID_x'].astype(str) \
#            + '_' + df_X['TeamID_y'].astype(str)

# df_X

In [7]:
# # Create Groundtruth data y
# df_y = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv")

# # Ensure the smaller TeamID comes first
# df_y["Team1"] = df_y[["WTeamID", "LTeamID"]].min(axis=1)
# df_y["Team2"] = df_y[["WTeamID", "LTeamID"]].max(axis=1)

# # Create the ID column in the format 'Season_Team1_Team2'
# df_y["ID"] = df_y["Season"].astype(str) + "_" + y_true["Team1"].astype(str) + "_" + y_true["Team2"].astype(str)

# # Compute the margin (score difference) and adjust if TeamIDs were swapped
# df_y["Margin"] = df_y.apply(lambda row: row["WScore"] - row["LScore"] if row["WTeamID"] == row["Team1"] else row["LScore"] - row["WScore"], axis=1)
# df_y['W/L'] = (df_y['Margin'] > 0).astype(int)

# df_y

In [8]:
# # train data
# df_train = pd.merge(left=df_y, right=df_X,
#                     on='ID', how='inner')
# df_final = df_train[['ID', 'Season_x', 'Margin', 'W/L',  
#                      'TeamID_x', 'TeamID_y', 
#                      'Win%_x', 'margin_x', 'Score_x',
#                      'Win%_y', 'margin_y', 'Score_y',
#                      'FG%_x', '3P%_x', 'FT%_x', 'FG%_y', '3P%_y', 'FT%_y',
#                      'FGM_x', 'FGA_x', 'FGM3_x', 'FGA3_x', 'FTM_x', 'FTA_x', 'OR_x', 'DR_x', 'Ast_x', 'TO_x', 'Stl_x', 'Blk_x', 'PF_x',
#                      'FGM_y', 'FGA_y', 'FGM3_y', 'FGA3_y', 'FTM_y', 'FTA_y', 'OR_y', 'DR_y', 'Ast_y', 'TO_y', 'Stl_y', 'Blk_y', 'PF_y'
#                     ]]
# df_final.rename(columns={'Season_x':"Season"}, inplace=True)
# df_final.sort_values(by='ID', inplace=True)
# df_final

## 5. Validation

### Build a Model

In [9]:
# import xgboost as xgb
# import pandas as pd
# from sklearn.metrics import brier_score_loss

# # Define features and target
# features = [col for col in df_final.columns if col not in ["ID", "Season", "Margin", "W/L"]]

# # Splitting data into train and test sets based on Season
# X_train = df_final[df_final.Season < 2024][features]
# y_train = df_final[df_final.Season < 2024]["W/L"]
# X_test = df_final[df_final.Season == 2024][features]
# y_test = df_final[df_final.Season == 2024]["W/L"]

# # Initialize and train XGBoost Classifier
# xgb

# xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', 
#                                    n_estimators=1000, 
#                                    learning_rate=0.05, 
#                                    max_depth=10,
#                                    random_state=42)
# xgb_classifier.fit(X_train, y_train)

# # Get probability predictions for Brier Score
# y_pred_proba = xgb_classifier.predict_proba(X_test)[:, 1]

# # Compute Brier Score Loss
# brier_score = brier_score_loss(y_test, y_pred_proba)
# print(f"Brier Score Loss: {brier_score:.4f}")

## 6. Women's Tournament

## 7. Submission

In [10]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import brier_score_loss

# Load Regular Season Results
def load_regular_season_results(compact_results_url, detailed_results_url):
    regular_season_results = pd.read_csv(compact_results_url)
    regular_season_results['margin'] = regular_season_results['WScore'] - regular_season_results['LScore']
    rs_detailed = pd.read_csv(detailed_results_url)
    return pd.merge(regular_season_results, rs_detailed, on=['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WScore', 'LScore', 'WLoc', 'NumOT'], how='inner')

# Prepare Regular Season Data
def prepare_regular_season_data(df):
    W_columns = {'WTeamID': 'TeamID', 'WScore': 'Score', 'WLoc': 'Loc', 'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA',
                 'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast', 'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF'}
    L_columns = {'LTeamID': 'TeamID', 'LScore': 'Score', 'WLoc': 'Loc', 'LFGM': 'FGM', 'LFGA': 'FGA', 'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA',
                 'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast', 'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF'}
    
    W_results = df[['Season', 'DayNum', 'margin'] + list(W_columns.keys())].rename(columns=W_columns)
    L_results = df[['Season', 'DayNum', 'margin'] + list(L_columns.keys())].rename(columns=L_columns)
    W_results['Result'], L_results['Result'] = 1, 0
    L_results['margin'] = -L_results['margin']
    
    df_flatten = pd.concat([W_results, L_results], axis=0).drop('Loc', axis=1).reset_index(drop=True)
    summary_mean = df_flatten.groupby(['Season', 'TeamID'], as_index=False).mean()
    summary_mean['FG%'] = summary_mean['FGM'] / summary_mean['FGA']
    summary_mean['3P%'] = summary_mean['FGM3'] / summary_mean['FGA3']
    summary_mean['FT%'] = summary_mean['FTM'] / summary_mean['FTA']
    summary_mean.rename(columns={'Result': 'Win%'}, inplace=True)
    
    # Self join to create a Cartesian product with TeamID ordering
    df_cartesian = summary_mean.merge(summary_mean, on='Season', suffixes=('_x', '_y'))
    df_cartesian = df_cartesian[df_cartesian['TeamID_x'] < df_cartesian['TeamID_y']]
    
    # Create ID variable
    df_cartesian['ID'] = df_cartesian['Season'].astype(str) + "_" + df_cartesian['TeamID_x'].astype(str) + "_" + df_cartesian['TeamID_y'].astype(str)
    
    return df_cartesian

# Load and Process NCAA Tournament Data
def load_tournament_results(tournament_results_url):
    df_y = pd.read_csv(tournament_results_url)
    df_y["Team1"] = df_y[["WTeamID", "LTeamID"]].min(axis=1)
    df_y["Team2"] = df_y[["WTeamID", "LTeamID"]].max(axis=1)
    df_y["ID"] = df_y["Season"].astype(str) + "_" + df_y["Team1"].astype(str) + "_" + df_y["Team2"].astype(str)
    df_y["Margin"] = df_y.apply(lambda row: row["WScore"] - row["LScore"] if row["WTeamID"] == row["Team1"] else row["LScore"] - row["WScore"], axis=1)
    df_y['W/L'] = (df_y['Margin'] > 0).astype(int)
    return df_y

# Merge Training Data
def merge_training_data(df_X, df_y):
    df_train = pd.merge(df_y, df_X, on='ID', how='inner')
    df_final = df_train.rename(columns={'Season_x': "Season"}).sort_values(by='ID')
    return df_final

# Train and Evaluate Model
def train_and_evaluate(df_final, features):
    X_train, y_train = df_final[df_final.Season < 2024][features], df_final[df_final.Season < 2024]["Margin"]
    X_test, y_test = df_final[df_final.Season == 2024][features], df_final[df_final.Season == 2024]["W/L"]
    
    model = xgb.XGBRegressor(objective='reg:squarederror', 
                             n_estimators=1000, 
                             learning_rate=0.05, 
                             max_depth=10, 
                             random_state=42)
    model.fit(X_train, y_train)
    
    y_pred_margin = model.predict(X_test)
    y_pred_proba = 1 / (1 + np.exp(-y_pred_margin / 10))  # Scaling factor can be tuned
    
    brier_score = brier_score_loss(y_test, y_pred_proba)
    print(f"Brier Score Loss: {brier_score:.4f}")
    
    # Create prediction dataframe
    model.fit(df_final[features], df_final['W/L'])
    
    df_prediction = df_final[df_final.Season == 2025][["ID"]].copy()
    df_prediction["Pred"] = 1 / (1 + np.exp(-model.predict(df_final[df_final.Season == 2025][features]) / 10))
    
    return df_prediction

# Define Features
features = [
    "Win%_x", "margin_x", "Score_x", "FG%_x", "3P%_x", "FT%_x", "OR_x", "DR_x", "Ast_x", "TO_x", "Stl_x", "Blk_x", "PF_x",
    "Win%_y", "margin_y", "Score_y", "FG%_y", "3P%_y", "FT%_y", "OR_y", "DR_y", "Ast_y", "TO_y", "Stl_y", "Blk_y", "PF_y"
]

# Men's prediction
compact_results_url = '/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv'
detailed_results_url = '/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv'
tournament_results_url = '/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv'

df_season = load_regular_season_results(compact_results_url, detailed_results_url)
df_summary = prepare_regular_season_data(df_season)
df_tourney = load_tournament_results(tournament_results_url)
df_final = merge_training_data(df_summary, df_tourney)
df_prediction_men = train_and_evaluate(df_final, features)

# Women's Prediction
compact_results_url = '/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv'
detailed_results_url = '/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv'
tournament_results_url = '/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv'

# Define URLs
df_season = load_regular_season_results(compact_results_url, detailed_results_url)
df_summary = prepare_regular_season_data(df_season)
df_tourney = load_tournament_results(tournament_results_url)
df_final = merge_training_data(df_summary, df_tourney)
df_prediction_women = train_and_evaluate(df_final, features)

# Create final submission file
submission_df = pd.concat([
    df_prediction_men,
    df_prediction_women
], axis=0).sort_values(by='ID')

# Save submission file
submission_df.to_csv("submission.csv", index=False)
print("Submission file created successfully.")

Brier Score Loss: 0.2085
Brier Score Loss: 0.1931
Submission file created successfully.



## References

- [Simple starter notebook for March Mania 2025](https://www.kaggle.com/code/paultimothymooney/simple-starter-notebook-for-march-mania-2025)
- [What is a Brier Score and How is it Calculated?](https://www.cultivatelabs.com/crowdsourced-forecasting-guide/what-is-a-brier-score-and-how-is-it-calculated)
